In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 23 16:41:29 2018

@author: talk2
"""

import random

SUITS = ['Hearts', 'Diamonds','Clubs','Spades']
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
MAX_BALANCE = 750
CHIP_BALANCE = 0


class Card(object):
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

        if rank == 'A':
            self.point = 11
        elif rank in ['K', 'Q', 'J']:
            self.point = 10
        else:
            self.point = int(rank)

        self.hidden = False

    def __str__(self):
        if self.hidden:
            return '[X]'
        else:
            return '[' + self.suit + ' ' + self.rank + ']'

    def hide_card(self):
        self.hidden = True

    def reveal_card(self):
        self.hidden = False

    def is_ace(self):
        return self.rank == 'A'


class Deck(object):
    def __init__(self):
        self.cards = [Card(suit, rank) for suit in SUITS for rank in RANKS]
        self.shuffle()

    def __str__(self):
        cards_in_deck = ''
        for card in self.cards:
            cards_in_deck = cards_in_deck + str(card) + ' '
        return cards_in_deck

    def shuffle(self):
        random.shuffle(self.cards)

    def deal_card(self):
        card = self.cards.pop(0)
        return card


class Hand(object):
    def __init__(self):
        self.hand = []

    def add_card(self, card):
        self.hand.append(card)
        return self.hand

    def get_value(self):
        aces = 0
        value = 0
        for card in self.hand:
            if card.is_ace():
                aces += 1
            value += card.point
        while (value > 21) and aces:
            value -= 10
            aces -= 1
        return value


class Dealer(Hand):
    def __init__(self, name, deck):
        Hand.__init__(self)
        self.name = name
        self.deck = deck
        self.isBust = False

    def show_hand(self):
        for card in self.hand:
            print (card,end='')
        print

    def hit(self):
        print ("Hitting...")
        self.add_card(self.deck.deal_card())
        return self.hand

    def stand(self):
        print ("%s gets %d. Done." % (self.name, self.get_value()))

    def check_bust(self):
        if self.get_value() > 21:
            self.isBust = True
            print ("%s gets bust!" % self.name)
        else:
            self.stand()


class Player(Dealer):
    def __init__(self, name, deck, bet):
        Dealer.__init__(self, name, deck)
        self.bet = bet
        self.isBust = False
        


def play(player, deck):
    print ('\n',player.name + ':')
    player.show_hand()
    if player.name == 'Dealer':
        while player.get_value() < 17:
            player.hit()
            player.show_hand()
        player.check_bust()
    else:
        global CHIP_BALANCE
        if CHIP_BALANCE > player.bet:
            if player.hand[0].point == player.hand[1].point:
                choice = input_func("Hit or Stand? (h/s): ", str.lower,
                                    range_=('h', 's'))
            else:
                choice = input_func("Hit or Stand? (h/s): ", str.lower,
                                    range_=('h', 's'))
        else:
            choice = input_func("Hit or Stand (h/s): ", str.lower, range_=('h', 's'))
        while choice == 'h':
            player.hit()
            player.show_hand()
            if player.get_value() > 21:
                player.isBust = True
                print ("%s gets bust!" % player.name)
                break
            choice = input_func("Hit or Stand? (h/s): ", str.lower, range_=('h', 's'))

        if choice == 's':
            player.stand()

    


def input_func(prompt, type_=None, min_=None, max_=None, range_=None):
    value = ''
    while True:
        value = input(prompt)
        if type_ is not None:
            try:
                value = type_(value)
            except ValueError:
                print ("Sorry I don't understand.")
                continue
        if min_ is not None and value < min_:
            print ("Sorry your input can not be less than %d!" % min_)
        elif max_ is not None and value > max_:
            print ("Sorry your input can not be more than %d!" % max_)
        elif range_ is not None and value not in range_:
            print ("You must select from", range_)
        else:
            break
    return value


def report(player, dealer):
    global CHIP_BALANCE
    if player.isBust:
        tag = 'Busted'
    elif dealer.isBust or (player.get_value() > dealer.get_value()):
        tag = 'Winner'
        CHIP_BALANCE += player.bet * 2
    elif player.get_value() == dealer.get_value():
        tag = 'Tie'
        CHIP_BALANCE += player.bet
    else:
        tag = 'Busted'
    print("%s: %-*s Balance = %d" % (player.name, 10, tag, CHIP_BALANCE))

def game():
    players = []
    global CHIP_BALANCE
    deck = Deck()

    print ("\nLet's get started...\n")

    if CHIP_BALANCE > 0:
        player_name = 'Player' 
        print ("%s:" % player_name)
        player_bet = input_func("Please bet. The minimal bet is 1 chip. ", int, 1, CHIP_BALANCE)
        CHIP_BALANCE -= player_bet
        print ("Balance updated. New balance is %d." % CHIP_BALANCE)
        player = Player(player_name, deck, player_bet)
        players.append(player)
        

    dealer = Dealer('Dealer', deck)

    for i in range(2):
        for player in (players + [dealer]):
            player.add_card(deck.deal_card())

    dealer.hand[1].hide_card()
    print ("\nDealer:")
    dealer.show_hand()
    print
    dealer.hand[1].reveal_card()

    for player in (players + [dealer]):
        play(player, deck)
        print

    print ("\nFinal result is:\n")

    for player in players:
        report(player, dealer)
    

    print( "\nFinal chip balance is %d.\n" % CHIP_BALANCE)


if __name__ == '__main__':

    CHIP_BALANCE = input_func("\nWelcome to BlackJack! Please enter the chip balance: (1-750) ", int, 1, MAX_BALANCE)
    while True:
        game()
        if CHIP_BALANCE < 1:
            print ("You don't have enough balance to proceed. Game over.")
            break
        proceed = input_func("Do you want to continue? (y/n) ", str.lower, range_=('y', 'n'))
        if proceed == 'n':
            print ("\nThanks for playing! Hope to see you soon!!!")
            break



Welcome to BlackJack! Please enter the chip balance: (1-750) 500

Let's get started...

Player:
Please bet. The minimal bet is 1 chip. 50
Balance updated. New balance is 450.

Dealer:
[Spades Q][X]
 Player:
[Clubs 9][Clubs 6]Hit or Stand? (h/s): h
Hitting...
[Clubs 9][Clubs 6][Diamonds 7]Player gets bust!

 Dealer:
[Spades Q][Hearts 7]Dealer gets 17. Done.

Final result is:

Player: Busted     Balance = 450

Final chip balance is 450.

Do you want to continue? (y/n) n

Thanks for playing! Hope to see you soon!!!
